# 何故regexで国名抽出するときにNERで前処理する必要があるか？

Date: 2023/12/06, 2023/12/07

国名リストがある場合、spaCyでNERを行い、その結果に対して国名抽出すると良い。

- 固有表現、かつ、GPE, ORG, LOC であれば、国を意味する表現から国名抽出している確率が高くなる。
- NERかけないと、国を意味しない表現から国名抽出してしまうケースが多く発生してしまう。

In [1]:
import spacy
from spacy.matcher import PhraseMatcher
import re

nlp = spacy.load('ja_core_news_sm')

In [2]:
TEXT1 = '明日は本社で日本語を話せるドイツ人に合う'
TEXT2 = '明日は本社でドイツの海外現地法人から来た出張者と日本の話をする'
TEXT3 = '昨日はメンタイコでご飯を食べた'
TEXT4 = 'コンゴ共和国にはゴリラがいるのか？'
TEXT5 = 'コンゴにはゴリラがいるのか？'
pattern = r'日本|ドイツ|タイ|コンゴ|コンゴ共和国'
pattern_for_nre = r'^日本$|^ドイツ$|^タイ$|コンゴ'
pattern_for_nre2 = r'^日本$|^ドイツ$|^タイ$|^コンゴ共和国$'
pattern_for_phasematcher = ['日本', 'ドイツ', 'タイ', 'コンゴ共和国']
pattern_for_phasematcher = [nlp.make_doc(term) for term in pattern_for_phasematcher]

## NERなしの国名認識が失敗する例

In [3]:
for text in (TEXT1, TEXT2, TEXT3, TEXT4, TEXT5):
    print(f'---{text}')
    for m in re.finditer(pattern, text):
        print(f'Match found: {text[m.start():m.end()]} {m.group(0)}')

---明日は本社で日本語を話せるドイツ人に合う
Match found: 日本 日本
Match found: ドイツ ドイツ
---明日は本社でドイツの海外現地法人から来た出張者と日本の話をする
Match found: ドイツ ドイツ
Match found: 日本 日本
---昨日はメンタイコでご飯を食べた
Match found: タイ タイ
---コンゴ共和国にはゴリラがいるのか？
Match found: コンゴ コンゴ
---コンゴにはゴリラがいるのか？
Match found: コンゴ コンゴ


In [4]:
# Tokenでマッチをかければメンタイコがタイと認識されなくなるが、日本語が日本、ドイツ語がドイツと認識されてしまう。
# 製品名の辞書があって、文章から製品名を抽出するケースでは、このパターンでも良いかも。
for doc in nlp.pipe((TEXT3, TEXT1)):
    for token in doc:
        print(f'---{token}')
        match = re.match(pattern_for_nre2, token.text)
        if match:
            start = match.start()
            end = match.end()
            print(f'Match found: {token.text[start:end]}')  

---昨日
---は
---メンタイコ
---で
---ご飯
---を
---食べ
---た
---明日
---は
---本社
---で
---日本
Match found: 日本
---語
---を
---話せる
---ドイツ
Match found: ドイツ
---人
---に
---合う


In [5]:
#PhraseMatcherを使っても、日本語が日本、ドイツ語がドイツと認識されてしまう。
# 製品名の辞書があって、文章から製品名を抽出するケースでは、PhaseMatcher使えるかも。
matcher = PhraseMatcher(nlp.vocab)
matcher.add('countries', pattern_for_phasematcher)

for doc in nlp.pipe((TEXT1, TEXT2, TEXT3, TEXT4, TEXT5)):
    print(f'---{doc.text}')
    matches = matcher(doc, as_spans=True)
    for span in matches:
        print(span.text, span.label_)

---明日は本社で日本語を話せるドイツ人に合う
日本 countries
ドイツ countries
---明日は本社でドイツの海外現地法人から来た出張者と日本の話をする
ドイツ countries
日本 countries
---昨日はメンタイコでご飯を食べた
---コンゴ共和国にはゴリラがいるのか？
コンゴ共和国 countries
---コンゴにはゴリラがいるのか？


## NERありの国名認識では失敗しない

In [6]:
# 注意：正規表現を^...$で囲わないと部分マッチになる
for doc in nlp.pipe([TEXT1, TEXT2, TEXT3, TEXT4, TEXT5]):
    print(f'---{doc.text}')
    for ent in doc.ents:
        #print(ent.text, ent.label_)
        if ent.label_ in ('GPE', 'ORG', 'LOC'):
            print(ent.text)
            match = re.match(pattern_for_nre, ent.text)
            if match:
                start = match.start()
                end = match.end()
                print(f'Match found: {ent[start:end]} {ent.label_}')

---明日は本社で日本語を話せるドイツ人に合う
---明日は本社でドイツの海外現地法人から来た出張者と日本の話をする
ドイツ
Match found: ドイツ GPE
日本
Match found: 日本 GPE
---昨日はメンタイコでご飯を食べた
---コンゴ共和国にはゴリラがいるのか？
コンゴ共和国
Match found: コンゴ共和国 GPE
---コンゴにはゴリラがいるのか？


In [7]:
# 注意：正規表現を^...$で囲うと完全マッチになる
for doc in nlp.pipe([TEXT1, TEXT2, TEXT3, TEXT4, TEXT5]):
    print(f'---{doc.text}')
    for ent in doc.ents:
        #print(ent.text, ent.label_)
        if ent.label_ in ('GPE', 'ORG', 'LOC'):
            print(ent.text)
            match = re.match(pattern_for_nre2, ent.text)
            if match:
                start = match.start()
                end = match.end()
                print(f'Match found: {ent[start:end]} {ent.label_}')

---明日は本社で日本語を話せるドイツ人に合う
---明日は本社でドイツの海外現地法人から来た出張者と日本の話をする
ドイツ
Match found: ドイツ GPE
日本
Match found: 日本 GPE
---昨日はメンタイコでご飯を食べた
---コンゴ共和国にはゴリラがいるのか？
コンゴ共和国
Match found: コンゴ共和国 GPE
---コンゴにはゴリラがいるのか？
